## Replace doi references with actual links

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd
from collections import Counter

In [4]:
from urllib.parse import urlparse

In [5]:
from functions import log, isInRef

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
headers = { # this is my header
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [8]:
# dictionary for getting journal name from url
journals_df = pd.read_csv('domain_to_journal.csv', delimiter='\t')
journal_ref = pd.Series(journals_df.journal.values,index=journals_df.domain).to_dict()
# dict: journal_ref['url'] = journal name

In [9]:
df = pd.read_csv('results.2.csv', delimiter= '\t')

In [10]:
df.head()

,link,journal_domain,title,abstract,references,doi
0,https://doi.org/10.1371/journal.pone.0174944,journals.plos.org,Can machine-learning improve cardiovascular ri...,Current approaches to predict cardiovascular r...,['https://doi.org/10.1136/bmj.39609.449676.25'...,https://doi.org/10.1371/journal.pone.0174944
1,https://doi.org/10.1371/journal.pone.0181142,journals.plos.org,"""What is relevant in a text document?"": An int...",Text documents can be described by a number of...,"['https://doi.org/10.1108/eb026526', 'https://...",https://doi.org/10.1371/journal.pone.0181142
2,https://doi.org/10.1371/journal.pone.0164270,journals.plos.org,Reversible Cryopreservation of Living Cells Us...,Rapid cooling of aqueous solutions is a useful...,"['https://doi.org/10.1038/164666a0', 'https://...",https://doi.org/10.1371/journal.pone.0164270
3,https://doi.org/10.1371/journal.pone.0144916,journals.plos.org,Improving Cycling Performance: Transcranial Di...,The central nervous system seems to have an im...,['https://doi.org/10.1152/japplphysiol.91324.2...,https://doi.org/10.1371/journal.pone.0144916
4,https://doi.org/10.1371/journal.pone.0089157,journals.plos.org,Changes in Voluntary Activation Assessed by Tr...,Maximal central motor drive is known to decrea...,[],https://doi.org/10.1371/journal.pone.0089157


In [11]:
# # all references in flat list
# all_references = []
# for sublist in df.references:
#     sub = eval(sublist) # convert from string
#     for item in sub:        
#         all_references.append(item)    

In [12]:
# initialize journal list
journals = []

In [13]:
# collect an example link for every journal
samples = {}
def get_sample_url(url, domain):
    if not(domain in samples.keys()):
        samples[domain] = url

In [14]:
def main():
    # will replace references column
    all_references = []
    counter = 0; # for knowing progress
    for sublist in df.references:
        sub_references = []
        sub = eval(sublist) # bc its stored as string, parse to list     
        for link in sub:
            try:
                # get request and extract domain
                request = requests.get(link, headers=headers, verify=False, timeout = 5)
                domain = urlparse(request.url).netloc
                # append to master list of domains
                journals.append(domain)
                # collect example article for each domain
                get_sample_url(request.url, domain)
                # check if domain is in reference file as some journal
                if isInRef(domain):            
                    # append to list of references for this article
                    sub_references.append(link+"|"+domain)

                    # counter for progress purposes
                    counter+=1;
                    if counter%25 == 0:
                        print(counter)
                else:
                    # do same thing actually lol
                    sub_references.append(link+"|"+domain)

                    # counter for progress purposes
                    counter+=1;
                    if counter%25 == 0:
                        print(counter)
                        
            except:
                print("Timeout occurred")
                    
                
        # append list of references for this article        
        all_references.append(sub_references)
        
    df.references = all_references
    
    log(f'Ran analyze_ref.3.ipynb succesfully')


In [15]:
main()

25
50
75
100
125
Timeout occurred


In [16]:
df.head()

,link,journal_domain,title,abstract,references,doi
0,https://doi.org/10.1371/journal.pone.0174944,journals.plos.org,Can machine-learning improve cardiovascular ri...,Current approaches to predict cardiovascular r...,[https://doi.org/10.1136/bmj.39609.449676.25|w...,https://doi.org/10.1371/journal.pone.0174944
1,https://doi.org/10.1371/journal.pone.0181142,journals.plos.org,"""What is relevant in a text document?"": An int...",Text documents can be described by a number of...,[https://doi.org/10.1108/eb026526|www.emerald....,https://doi.org/10.1371/journal.pone.0181142
2,https://doi.org/10.1371/journal.pone.0164270,journals.plos.org,Reversible Cryopreservation of Living Cells Us...,Rapid cooling of aqueous solutions is a useful...,[https://doi.org/10.1038/164666a0|www.nature.c...,https://doi.org/10.1371/journal.pone.0164270
3,https://doi.org/10.1371/journal.pone.0144916,journals.plos.org,Improving Cycling Performance: Transcranial Di...,The central nervous system seems to have an im...,[https://doi.org/10.1152/japplphysiol.91324.20...,https://doi.org/10.1371/journal.pone.0144916
4,https://doi.org/10.1371/journal.pone.0089157,journals.plos.org,Changes in Voluntary Activation Assessed by Tr...,Maximal central motor drive is known to decrea...,[],https://doi.org/10.1371/journal.pone.0089157


In [17]:
df.to_csv(r'results.3.csv', sep='\t', index = False)

In [18]:
len(journals)

137

In [19]:
counter = Counter(journals)

In [20]:
counter.most_common()

[('linkinghub.elsevier.com', 42),
 ('onlinelibrary.wiley.com', 16),
 ('link.springer.com', 12),
 ('journals.plos.org', 8),
 ('journals.lww.com', 7),
 ('academic.oup.com', 6),
 ('www.nature.com', 6),
 ('www.nejm.org', 5),
 ('ieeexplore.ieee.org', 5),
 ('royalsocietypublishing.org', 4),
 ('jamanetwork.com', 2),
 ('heart.bmj.com', 2),
 ('www.cambridge.org', 2),
 ('journals.physiology.org', 2),
 ('journals.sagepub.com', 2),
 ('www.science.org', 2),
 ('bmcecolevol.biomedcentral.com', 2),
 ('www.tandfonline.com', 2),
 ('validate.perfdrive.com', 2),
 ('www.bmj.com', 1),
 ('www.ahajournals.org', 1),
 ('www.emerald.com', 1),
 ('dl.acm.org', 1),
 ('ecerm.org', 1),
 ('www.hindawi.com', 1),
 ('ijbnpa.biomedcentral.com', 1),
 ('n.neurology.org', 1)]

In [21]:
samples

{'www.bmj.com': 'https://www.bmj.com/content/336/7659/1475',
 'www.ahajournals.org': 'https://www.ahajournals.org/doi/10.1161/CIRCULATIONAHA.107.699579',
 'jamanetwork.com': 'https://jamanetwork.com/journals/jama/fullarticle/205528',
 'www.nejm.org': 'https://www.nejm.org/doi/full/10.1056/NEJMoa0807646',
 'linkinghub.elsevier.com': 'https://linkinghub.elsevier.com/retrieve/pii/S0738399112004491',
 'onlinelibrary.wiley.com': 'https://onlinelibrary.wiley.com/doi/10.1111/j.1365-2125.2009.03537.x',
 'heart.bmj.com': 'https://heart.bmj.com/content/90/8/859',
 'academic.oup.com': 'https://academic.oup.com/bioinformatics/article/32/16/2411/2240358',
 'journals.lww.com': 'https://journals.lww.com/lww-medicalcare/Abstract/2013/03000/Improved_Cardiovascular_Risk_Prediction_Using.7.aspx',
 'www.emerald.com': 'https://www.emerald.com/insight/content/doi/10.1108/eb026526/full/html',
 'dl.acm.org': 'https://dl.acm.org/doi/10.1145/361219.361220',
 'www.nature.com': 'https://www.nature.com/articles/nc